# Feature Engineering

Since my main focus in this project is to compare Neural Networks with classical ML models. I will not focus that much on this section and I will just do some basic things.
1. I think there is some space for creating new features
2. After creating new features I will check feature importance and eventually remove some
3. At the end I will split the data into train and test and use created before pipeline.
4. split train into train and validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer
from tensorflow.keras import backend as K
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from sklearn.feature_selection import RFE

In [8]:
dataset_df = pd.read_csv(r'C:\Users\Adam\Desktop\main\programming\machine learning\house prices regression\data\preprocessed\entire_prep_dataset.csv')
train = pd.read_csv(r"C:\Users\Adam\Desktop\main\programming\machine learning\house prices regression\data\raw\train.csv")
test = pd.read_csv(r"C:\Users\Adam\Desktop\main\programming\machine learning\house prices regression\data\raw\test.csv")
dataset_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,4.110874,RL,4.189655,9.042040,Pave,Reg,Lvl,AllPub,Inside,...,4.127134,0.000000,0.0,0.0,0.0,0.0,2,2008,WD,Normal
1,2,3.044522,RL,4.394449,9.169623,Pave,Reg,Lvl,AllPub,FR2,...,0.000000,0.000000,0.0,0.0,0.0,0.0,5,2007,WD,Normal
2,3,4.110874,RL,4.234107,9.328212,Pave,IR1,Lvl,AllPub,Inside,...,3.761200,0.000000,0.0,0.0,0.0,0.0,9,2008,WD,Normal
3,4,4.262680,RL,4.110874,9.164401,Pave,IR1,Lvl,AllPub,Corner,...,3.583519,5.609472,0.0,0.0,0.0,0.0,2,2006,WD,Abnorml
4,5,4.110874,RL,4.442651,9.565284,Pave,IR1,Lvl,AllPub,FR2,...,4.442651,0.000000,0.0,0.0,0.0,0.0,12,2008,WD,Normal


In [108]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [109]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [9]:
# split the data for setting threshold
train_modified = dataset_df[:train.shape[0]].copy()
test_modified = dataset_df[train.shape[0]:].copy()
#train_modified.shape, test_modified.shape

# for test dataframe we need to add SalePrice feature that we have dropped before
train_modified['SalePrice'] = train['SalePrice']
train_modified.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,4.110874,RL,4.189655,9.042040,Pave,Reg,Lvl,AllPub,Inside,...,0.000000,0.0,0.0,0.0,0.0,2,2008,WD,Normal,208500
1,2,3.044522,RL,4.394449,9.169623,Pave,Reg,Lvl,AllPub,FR2,...,0.000000,0.0,0.0,0.0,0.0,5,2007,WD,Normal,181500
2,3,4.110874,RL,4.234107,9.328212,Pave,IR1,Lvl,AllPub,Inside,...,0.000000,0.0,0.0,0.0,0.0,9,2008,WD,Normal,223500
3,4,4.262680,RL,4.110874,9.164401,Pave,IR1,Lvl,AllPub,Corner,...,5.609472,0.0,0.0,0.0,0.0,2,2006,WD,Abnorml,140000
4,5,4.110874,RL,4.442651,9.565284,Pave,IR1,Lvl,AllPub,FR2,...,0.000000,0.0,0.0,0.0,0.0,12,2008,WD,Normal,250000


In [111]:
# get list of numeric features and categorical features
num_cols = train_modified.select_dtypes(include=['number']).columns.tolist()
cat_cols = train_modified.select_dtypes(exclude=['number']).columns.tolist()

len(num_cols), len(cat_cols)

(38, 39)

## 0. Threshold
I think that it is worth setting some threshold in order to see how changes impact model performence. For this I will train some baseline models.
* LinearRegression,
*  RandomForestRegressor,
*   DNN

In [7]:
# new pipeline
def my_pipeline(df):
    """
    Encodes non-numeric columns using OneHotEncoder and scales numeric columns using StandardScaler.

    Parameters:
        df (pd.DataFrame): Input dataset as a pandas DataFrame.

    Returns:
        pd.DataFrame: Transformed dataset with encoded and scaled features.
    """
    # Identify numeric and categorical columns
    numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
    categorical_features = df.select_dtypes(exclude=[np.number]).columns.tolist()

    # Define transformers for numeric and categorical features
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

    # Combine transformers in a ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    # Apply transformations
    transformed_data = preprocessor.fit_transform(df)

    # Get column names for transformed data
    cat_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
    all_columns = numeric_features + list(cat_columns)

    # Create a DataFrame with transformed data
    transformed_df = pd.DataFrame(transformed_data, columns=all_columns, index=df.index)

    return transformed_df

In [113]:
"""def my_pipeline(train, test):

    #This function prepares data for training and evaluating by applying preprocessing steps 
    #using a pipeline with OneHotEncoder and StandardScaler.
    
    #Parameters:
    #    train: DataFrame (includes 'SalePrice' column)
    #    test: DataFrame (does not include 'SalePrice' column)
        
    #Returns:
    #    Transformed train and test DataFrames
    #    lsit of modified features 

    X_train = train.drop(columns=['SalePrice'])
    y_train = train['SalePrice']
    X_test = test.copy()

    num_cols = X_train.select_dtypes(include=['number']).columns.tolist()
    cat_cols = X_train.select_dtypes(exclude=['number']).columns.tolist()

    # debug
    #print(f"Numeric columns: {num_cols}")
    #print(f"Categorical columns: {cat_cols}")
    
    # create transformers for numerical and categorical data
    num_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    cat_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

    # combine preprocessors in a ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_cols),
            ('cat', cat_transformer, cat_cols)
        ]
    )

    # fit and transform the data
    X_train_proc = preprocessor.fit_transform(X_train)
    #print(f"Shape of X_train_proc {X_train_proc.shape}")
    X_test_proc = preprocessor.transform(X_test)

    
    feature_names = []
    if cat_cols:
        feature_names = list(preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(cat_cols))
    print(feature_names)
    
    return X_train_proc, X_test_proc, feature_names
"""

'def my_pipeline(train, test):\n\n    #This function prepares data for training and evaluating by applying preprocessing steps \n    #using a pipeline with OneHotEncoder and StandardScaler.\n    \n    #Parameters:\n    #    train: DataFrame (includes \'SalePrice\' column)\n    #    test: DataFrame (does not include \'SalePrice\' column)\n        \n    #Returns:\n    #    Transformed train and test DataFrames\n    #    lsit of modified features \n\n    X_train = train.drop(columns=[\'SalePrice\'])\n    y_train = train[\'SalePrice\']\n    X_test = test.copy()\n\n    num_cols = X_train.select_dtypes(include=[\'number\']).columns.tolist()\n    cat_cols = X_train.select_dtypes(exclude=[\'number\']).columns.tolist()\n\n    # debug\n    #print(f"Numeric columns: {num_cols}")\n    #print(f"Categorical columns: {cat_cols}")\n    \n    # create transformers for numerical and categorical data\n    num_transformer = Pipeline(steps=[(\'scaler\', StandardScaler())])\n    cat_transformer = Pipeline(s

In [114]:
#train_proc, test_proc, tansformed_cols = my_pipeline(train_modified, test_modified)

In [115]:
#len(tansformed_cols), train_proc.shape

In [116]:
#target_col = train['SalePrice']
#X_train, X_val, y_train, y_val = train_test_split(train_proc,
#                                                   target_col,
#                                                   test_size=0.2,
#                                                   random_state=42)

In [117]:
def rmse(y_true, y_preds):
    y_true = K.cast(y_true, dtype='float32')
    y_preds = K.cast(y_preds, dtype='float32')
    return K.sqrt(K.mean(K.square(y_true - y_preds)))

def DNN(X_train, y_train, X_val, y_val):
    """
    DNN is just a simple dense neural network. It is ment to make a simple evaluation of changes, modifications. NOT for further complex evaluations!
    Parameters:
        X_train, y_train, X_val, y_val - splitted dataset
    Returns:
        It returns 4 parameteres:
        train_loss- loss on the train data
        train_rmse- rmse score on the train data
        val_loss- loss on the validation data 
        val_rmse- rmse score on the validation data
    """
    model = Sequential([
        Dense(64, activation='relu', input_dim=X_train.shape[1]),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=[rmse])

    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=0)
    
    train_loss, train_rmse = model.evaluate(X_train, y_train, verbose=0)
    val_loss, val_rmse = model.evaluate(X_val, y_val, verbose=0)
    return train_loss, train_rmse, val_loss, val_rmse

In [118]:
#%%time
#train_loss, train_rmse, val_loss, val_rmse = DNN(X_train, y_train, X_val, y_val)


In [119]:
#train_loss, train_rmse, val_loss, val_rmse

In [120]:
#threshold = {'DNN':val_rmse}
#threshold

In [121]:
def evaluate_models(X_train, X_val, y_train, y_val, models):
    """
    This method uses models (without tunning them) to measure performence after some changes. Metric used for evaluation is RMSE. 
    Parameters:
        X_train, X_val, y_train, y_val: ready to use, preprocessed data
        models: (dictionary) models to evaluate data on {"model_name": model()}

    Returns:
        returns lsit of models name and its rmse score on train and validation data. 
    """
    rmse_scores = []
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_preds_train = model.predict(X_train)
        y_preds_val = model.predict(X_val)
        rmse_train = rmse(y_train, y_preds_train)
        rmse_val = rmse(y_val, y_preds_val)
        rmse_scores.append([name, rmse_train, rmse_val])
    return rmse_scores

In [122]:
#%%time
#models = {'LinearRegression': LinearRegression(),
#          'RandomForestRegressor': RandomForestRegressor(random_state=42)}
#model_scores = evaluate_models(X_train, X_val, y_train, y_val, models)
#model_scores

In [123]:
#threshold['LinearRegression'] = np.array(model_scores[0][2]).item()
#threshold['RandomForestRegressor'] = np.array(model_scores[1][2]).item()
#threshold

In [124]:
#threshold_df = pd.DataFrame(list(threshold.items()), columns=['model', 'rmse'])
#threshold_df.to_csv('threshold.csv')
#threshold_df

In [125]:
def evaluate_changes(dataset_df, target, train_shape, models):
    """ 
    This method is connection of models evaluation and pipeline. It evaluates both given ML models and simple DNN.
    Parameters:
        dataset_df- (dataframe) train data (without target column) + test data
        target- SalePrice column
        train_shape- shape of the train data (without target column)
        test_shape-shape of the test data
        models- (dictiornaty) of models to evaluate on
    Returns:
        dataframe with models and their rmse scores
    """
    #train_df = dataset_df[:train_shape[0]].copy()
    #test_df = dataset_df[train_shape[0]:].copy()

    #train_df['SalePrice'] = targetb
    #train, test, _ = my_pipeline(train_df, test_df)

    #X_train, X_val, y_train, y_val = train_test_split(train,
    #                                                  target,
    #                                                  test_size=0.2,
    #                                                  random_state=42)
    preprocessed_dataset = my_pipeline(dataset_df)
    X = preprocessed_dataset[:train_shape[0]].copy()
    X_train, X_val, y_train, y_val = train_test_split(X,
                                                      target,
                                                      test_size=0.2,
                                                      random_state=42)
    rmse_scores_ml = evaluate_models(X_train, X_val, y_train, y_val, models)
    train_loss, train_rmse_dnn, val_loss, val_rmse_dnn = DNN(X_train, y_train, X_val, y_val)
    #dnn_info = [train_loss, train_rmse_dnn, val_loss, val_rmse_dnn]

    rmse_scores = {'DNN':val_rmse_dnn}
    for i in range(len(models)):
        rmse_scores[rmse_scores_ml[i][0]] = np.array(rmse_scores_ml[i][2]).item()
    rmse_df = pd.DataFrame(list(rmse_scores.items()), columns=['model',' rmse'] )
    return rmse_df

In [239]:
target_col = train_modified['SalePrice']
models = {'LinearRegression': LinearRegression(),
          'RandomForestRegressor': RandomForestRegressor(random_state=42)}
rmse_threshold = evaluate_changes(dataset_df,target_col, train_modified.shape, models)
rmse_threshold

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,34363.132812
1,LinearRegression,31510.705078
2,RandomForestRegressor,28452.984375


## 1. New features
* AgeOfHouse - this feature can be created by: YrSold - YearBuilt,
* YearsSinceRenov - how many years ago there was the renovation: YrSold-YearRemodAdd
* TotalBathNum - total number of bathrooms: FullBath + HalfBath + BsmtFullBath + BsmtHalfBath
* PorchSize - OpenPorchSF + EnclosedPorch + 3SSnPorch + ScreenPorch
* IsBasement - Does the building has basement? TotalBsmtSF > 0  True/False
* IsFirePlace - -,,- Fireplaces > 0
* IsGarage - -,,- GarageArea > 0



In [127]:
# 1.
dataset_df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodD

In [128]:
# AgeOfHouse- this feature will contain the information how old the house was when it was sold.
AgeOfHouse = dataset_df['YrSold'] - dataset_df['YearBuilt'] 
dataset_df['AgeOfHouse'] = AgeOfHouse
#dataset_df.columns


In [129]:
%%time
rmse_age_of_houses = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_age_of_houses

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


CPU times: total: 17.7 s
Wall time: 15.1 s


,model,rmse
0,DNN,33868.511719
1,LinearRegression,31554.585938
2,RandomForestRegressor,28780.226562


In [130]:
#YearsSinceRenov

YearsSinceRenov = dataset_df['YrSold'] - dataset_df['YearRemodAdd']
dataset_df['YearsSinceRenov'] = YearsSinceRenov
dataset_df =dataset_df.drop(columns=['AgeOfHouse'])

rmse_yr_from_renov = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_yr_from_renov

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,34046.460938
1,LinearRegression,31554.585938
2,RandomForestRegressor,28831.613281


In [131]:
#TotalBathNum =  FullBath + HalfBath + BsmtFullBath + BsmtHalfBath

TotalBathNum = dataset_df['FullBath'] + dataset_df['HalfBath'] + dataset_df['BsmtFullBath'] + dataset_df['BsmtHalfBath']
dataset_df = dataset_df.drop(columns=['YearsSinceRenov'])
dataset_df['TotalBathNum'] = TotalBathNum
dataset_df.head() 

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,TotalBathNum
0,1,4.110874,RL,4.189655,9.042040,Pave,Reg,Lvl,AllPub,Inside,...,0.000000,0.0,0.0,0.0,0.0,2,2008,WD,Normal,4.000000
1,2,3.044522,RL,4.394449,9.169623,Pave,Reg,Lvl,AllPub,FR2,...,0.000000,0.0,0.0,0.0,0.0,5,2007,WD,Normal,2.693147
2,3,4.110874,RL,4.234107,9.328212,Pave,IR1,Lvl,AllPub,Inside,...,0.000000,0.0,0.0,0.0,0.0,9,2008,WD,Normal,4.000000
3,4,4.262680,RL,4.110874,9.164401,Pave,IR1,Lvl,AllPub,Corner,...,5.609472,0.0,0.0,0.0,0.0,2,2006,WD,Abnorml,2.000000
4,5,4.110874,RL,4.442651,9.565284,Pave,IR1,Lvl,AllPub,FR2,...,0.000000,0.0,0.0,0.0,0.0,12,2008,WD,Normal,4.000000


In [132]:
rmse_totBath = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_totBath

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,34555.921875
1,LinearRegression,31554.585938
2,RandomForestRegressor,28837.529297


In [133]:
# PorchSize - OpenPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch
PorchSize = dataset_df['OpenPorchSF'] + dataset_df['EnclosedPorch'] + dataset_df['3SsnPorch'] + dataset_df['ScreenPorch']
dataset_df['PorchSize'] = PorchSize
dataset_df = dataset_df.drop(columns=['TotalBathNum'])

rmse_PorchSize = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_PorchSize 

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,34536.246094
1,LinearRegression,31554.585938
2,RandomForestRegressor,29332.156250


In [134]:
# IsBasement - Does the building has basement? TotalBsmtSF > 0 True/False

IsBasement = dataset_df['TotalBsmtSF'] > 0
dataset_df = dataset_df.drop(columns=['PorchSize'])
dataset_df['IsBasement'] = IsBasement

rmse_IsBasement = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_IsBasement 

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,33685.320312
1,LinearRegression,31510.705078
2,RandomForestRegressor,29403.662109


In [135]:
# IsFirePlace - -,,- Fireplaces > 0
IsFirePlace = dataset_df['Fireplaces'] > 0
dataset_df = dataset_df.drop(columns=['IsBasement'])
dataset_df['IsFirePlace'] = IsFirePlace

rmse_IsFirePlace = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_IsFirePlace 

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,33695.210938
1,LinearRegression,31554.585938
2,RandomForestRegressor,29791.001953


In [136]:
# IsGarage - -,,- GarageArea > 0
IsGarage = dataset_df['GarageArea'] > 0
dataset_df = dataset_df.drop(columns=['IsFirePlace'])
dataset_df['IsGarage'] = IsGarage

rmse_IsGarage = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_IsGarage 

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,35030.105469
1,LinearRegression,31554.585938
2,RandomForestRegressor,29312.332031


In [137]:
dataset_df = dataset_df.drop(columns=['IsGarage'])

### NOTE
As shown above some features (single impacet tested) had positive impact and some didn't. This of course might have happend becouse the way different models work with features. Shortly, LinearRegression assume a linear relationship between features between features and target. Features that introduce non-linearity may confuse model. Random forests may overvit to noise and DNNs are sensitive to the scale and distribution of features. 

In [138]:
# Lets see how this features combined together impact this models
new_features = {'YearsSinceRenov':YearsSinceRenov,
                'TotalBathNum':TotalBathNum,
                'PorchSize':PorchSize,
                'IsBasement':IsBasement,
                'IsFirePlace':IsFirePlace,
                'IsGarage':IsGarage}
for name, feature in new_features.items():
    dataset_df[name] = feature

In [139]:
rmse_all_features = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_all_features 

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,34699.113281
1,LinearRegression,31510.705078
2,RandomForestRegressor,29241.156250


In [140]:
rmse_all_features.to_csv('rmse_of_new_feature.csv')

### NOTE
There is a small positiv impact on the DNNs and LinearRegression

## 2. Useless features
Now I will seek for some redundancy in the dataset and try removing some features if they turn out redundant.
1. Try dropping some highly correlated features
2. Use RandomForestRegressor to calculate feature importance
3. RFE

### NOTE
As a remainder here are most correlated features:

- GarageCars   GarageArea      0.882475
- YearBuilt    GarageYrBlt     0.825667
- GrLivArea    TotRmsAbvGrd    0.825489
- TotalBsmtSF  1stFlrSF        0.819530
-OverallQual  SalePrice       0.790982

In [141]:
# 1. I think there is no sense in testing and dropping all of these features since they are still not that much correlated,
# but the first one might have sense
GarageCars = dataset_df['GarageCars']
dataset_df = dataset_df.drop(columns=['GarageCars'])
rmse_dropped_GaragCars = evaluate_changes(dataset_df, target_col, train_modified.shape, models)
rmse_dropped_GaragCars

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


,model,rmse
0,DNN,35374.351562
1,LinearRegression,31627.617188
2,RandomForestRegressor,28983.630859


In [142]:
# This one still might contain some key informations different than GarageArea
dataset_df['GarageCars'] = GarageCars

In [184]:
# 2.calculate feature importance 
preproc_df = my_pipeline(dataset_df)

X = preproc_df[:train_modified.shape[0]].copy()
y = target_col

model = RandomForestRegressor(random_state=42)
model.fit(X, y)
importances = model.feature_importances_
importances_df = pd.DataFrame({'Feature':X.columns, 'Importance':importances}).sort_values(by='Importance',ascending=False)
importances_df

,Feature,Importance
4,OverallQual,0.580950
16,GrLivArea,0.099681
12,TotalBsmtSF,0.034995
14,2ndFlrSF,0.030825
9,BsmtFinSF1,0.027702
...,...,...
104,Condition2_RRAe,0.000000
126,RoofMatl_ClyTile,0.000000
176,ExterCond_Po,0.000000
55,Utilities_AllPub,0.000000


In [187]:
least_important10 = importances_df['Feature'].tolist()[-10:]
least_important10

['Exterior2nd_CBlock',
 'Utilities_NoSeWa',
 'Condition1_RRNe',
 'Exterior1st_ImStucc',
 'Condition2_PosA',
 'Condition2_RRAe',
 'RoofMatl_ClyTile',
 'ExterCond_Po',
 'Utilities_AllPub',
 'HeatingQC_Po']

In [190]:
# Now since we have importances of our features we can see how models will behave without x of the least important by this ranking
X_reduced = X.drop(columns=least_important10)
X_train, X_val, y_train, y_val = train_test_split(X_reduced, target_col, test_size=0.2, random_state=42)
rmse_dnns = DNN(X_train, y_train, X_val, y_val)
rmse_dnns[-1]

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


35078.30859375

In [191]:
models_rmse = evaluate_models(X_train, X_val, y_train, y_val, models)
models_rmse

[['LinearRegression',
  <tf.Tensor: shape=(), dtype=float32, numpy=21403.466796875>,
  <tf.Tensor: shape=(), dtype=float32, numpy=42205.03515625>],
 ['RandomForestRegressor',
  <tf.Tensor: shape=(), dtype=float32, numpy=11003.970703125>,
  <tf.Tensor: shape=(), dtype=float32, numpy=28851.29296875>]]

### NOTE 
In order to speed up the process of seeking best features combination I will just move on to the RFE

In [198]:
# 3. RFE
X = preproc_df[:train_modified.shape[0]].copy()
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  target_col,
                                                  test_size=0.2,
                                                  random_state=42)
model = RandomForestRegressor(random_state=42)
rfe = RFE(estimator=model, n_features_to_select=100)
rfe.fit(X_train, y_train)
selected_features = X_train.columns[rfe.support_]
print('Selected features: ', selected_features)

Selected features:  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
       'ScreenPorch', 'PoolArea', 'MoSold', 'YrSold', 'YearsSinceRenov',
       'TotalBathNum', 'PorchSize', 'GarageCars', 'MSZoning_RL', 'MSZoning_RM',
       'LotShape_IR1', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Lvl',
       'LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_Inside',
       'LandSlope_Gtl', 'Neighborhood_CollgCr', 'Neighborhood_Crawfor',
       'Neighborhood_NoRidge', 'Neighborhood_Timber', 'RoofStyle_Gable',
       'RoofStyle_Hip', 'RoofMatl_CompShg', 'Exterior1st_Plywood',
       'Exterior1st_Wd Sdng', 'Exterior

In [200]:
rmse_w100 = DNN(X_train[selected_features], y_train, X_val[selected_features], y_val)
rmse_w100[-1]

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


36048.09375

In [201]:
rmse_w100_models = evaluate_models(X_train[selected_features], X_val[selected_features], y_train, y_val, models)
rmse_w100_models

[['LinearRegression',
  <tf.Tensor: shape=(), dtype=float32, numpy=28100.908203125>,
  <tf.Tensor: shape=(), dtype=float32, numpy=34133.09375>],
 ['RandomForestRegressor',
  <tf.Tensor: shape=(), dtype=float32, numpy=11132.7880859375>,
  <tf.Tensor: shape=(), dtype=float32, numpy=28840.375>]]

In [202]:
%%time
values_to_test = [13,26,52,78,130,195]
selected_features = {}
model = RandomForestRegressor(random_state=42)
for value in values_to_test:
    rfe = RFE(estimator=model, n_features_to_select=value)
    rfe.fit(X_train, y_train)
    selected_features[value] = X_train.columns[rfe.support_]

CPU times: total: 53min 51s
Wall time: 54min 16s


In [224]:
import pickle 
path = r"C:\Users\Adam\Desktop\main\programming\machine learning\house prices regression\data\preprocessed\data.pkl"
with open("feature_selection_v1.pkl", "wb") as path:
    pickle.dump(selected_features, path)

In [211]:
selected_features

{13: Index(['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'BsmtFinSF1',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'GarageYrBlt',
        'GarageArea', 'TotalBathNum', 'GarageCars'],
       dtype='object'),
 26: Index(['Id', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF',
        '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'FullBath', 'TotRmsAbvGrd',
        'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'MoSold',
        'YearsSinceRenov', 'TotalBathNum', 'PorchSize', 'GarageCars',
        'BsmtQual_Ex', 'KitchenQual_Gd'],
       dtype='object'),
 52: Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
        'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
        'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
        'GrLivArea', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', '

In [214]:
dnn_rmse_feature_selection = {}
for value, features in selected_features.items():
    rmse_dnn = DNN(X_train[features], y_train, X_val[features], y_val)
    dnn_rmse_feature_selection[value] = rmse_dnn[-1]
    rmse_dnn

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__i

In [215]:
dnn_rmse_feature_selection

{13: 57476.7890625,
 26: 50081.55859375,
 52: 40254.55078125,
 78: 36526.82421875,
 130: 34573.37109375,
 195: 34542.37109375}

In [222]:
rmse_models_feature_selection = {}
for value, features in selected_features.items():
    rmse_models = evaluate_models(X_train[features], X_val[features], y_train, y_val, models)
    rmse_models_feature_selection[str(value) + 'LinearRegression'] = float(rmse_models[0][2].numpy())
    rmse_models_feature_selection[str(value) + 'RandomForestRegressor'] = float(rmse_models[1][2].numpy())

In [223]:
rmse_models_feature_selection

{'13LinearRegression': 41182.92578125,
 '13RandomForestRegressor': 28927.125,
 '26LinearRegression': 37548.1171875,
 '26RandomForestRegressor': 27879.63671875,
 '52LinearRegression': 36409.69921875,
 '52RandomForestRegressor': 28241.31640625,
 '78LinearRegression': 33322.8125,
 '78RandomForestRegressor': 29174.26171875,
 '130LinearRegression': 34623.5625,
 '130RandomForestRegressor': 28614.11328125,
 '195LinearRegression': 33311.32421875,
 '195RandomForestRegressor': 28876.748046875}

In [225]:
%%time
values_to_test = [205,225,250,280]
selected_features_v2 = {}
model = RandomForestRegressor(random_state=42)
for value in values_to_test:
    rfe = RFE(estimator=model, n_features_to_select=value)
    rfe.fit(X_train, y_train)
    selected_features_v2[value] = X_train.columns[rfe.support_]

CPU times: total: 10min 44s
Wall time: 10min 50s


In [231]:
selected_features_v2

{205: Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
        'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
        ...
        'SaleType_CWD', 'SaleType_New', 'SaleType_WD', 'SaleCondition_Abnorml',
        'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial',
        'IsFirePlace_False', 'IsFirePlace_True', 'IsGarage_True'],
       dtype='object', length=205),
 225: Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
        'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
        ...
        'SaleType_WD', 'SaleCondition_Abnorml', 'SaleCondition_Family',
        'SaleCondition_Normal', 'SaleCondition_Partial', 'IsBasement_False',
        'IsFirePlace_False', 'IsFirePlace_True', 'IsGarage_False',
        'IsGarage_True'],
       dtype='object', length=225),
 250: Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
        'OverallCond', 'YearBuilt', 'YearRemodAdd', 

In [226]:
path = r"C:\Users\Adam\Desktop\main\programming\machine learning\house prices regression\data\preprocessed\data2.pkl"
with open("data_feature_selection2.pkl", "wb") as path:
    pickle.dump(selected_features_v2, path)

In [235]:
for value, features in selected_features_v2.items():
    rmse_dnn = DNN(X_train[features], y_train, X_val[features], y_val)
    dnn_rmse_feature_selection[value] = rmse_dnn[-1]
    rmse_dnn

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__i

In [236]:
dnn_rmse_feature_selection

{13: 57476.7890625,
 26: 50081.55859375,
 52: 40254.55078125,
 78: 36526.82421875,
 130: 34573.37109375,
 195: 34542.37109375,
 205: 35386.2265625,
 225: 34495.3671875,
 250: 35141.11328125,
 280: 34466.1875}

In [237]:
for value, features in selected_features_v2.items():
    rmse_models = evaluate_models(X_train[features], X_val[features], y_train, y_val, models)
    rmse_models_feature_selection[str(value) + 'LinearRegression'] = float(rmse_models[0][2].numpy())
    rmse_models_feature_selection[str(value) + 'RandomForestRegressor'] = float(rmse_models[1][2].numpy())
rmse_models_feature_selection

{'13LinearRegression': 41182.92578125,
 '13RandomForestRegressor': 28927.125,
 '26LinearRegression': 37548.1171875,
 '26RandomForestRegressor': 27879.63671875,
 '52LinearRegression': 36409.69921875,
 '52RandomForestRegressor': 28241.31640625,
 '78LinearRegression': 33322.8125,
 '78RandomForestRegressor': 29174.26171875,
 '130LinearRegression': 34623.5625,
 '130RandomForestRegressor': 28614.11328125,
 '195LinearRegression': 33311.32421875,
 '195RandomForestRegressor': 28876.748046875,
 '205LinearRegression': 33220.03515625,
 '205RandomForestRegressor': 29044.810546875,
 '225LinearRegression': 34066.734375,
 '225RandomForestRegressor': 28771.712890625,
 '250LinearRegression': 34811.43359375,
 '250RandomForestRegressor': 28817.09375,
 '280LinearRegression': 34947.8828125,
 '280RandomForestRegressor': 28246.1796875}

In [241]:
rmse_threshold

,model,rmse
0,DNN,34363.132812
1,LinearRegression,31510.705078
2,RandomForestRegressor,28452.984375


### NOTE 
Here we can see that there is a small improvement for a RandomForest, while other model's rmse went slightly down. At the end I would like to see if we can get some better scores if we run feature selection but on the pure dataset (without features that I have created).

In [244]:
pure_dataset = pd.read_csv(r'C:\Users\Adam\Desktop\main\programming\machine learning\house prices regression\data\preprocessed\entire_prep_dataset.csv')

In [245]:
preproc_dataset = my_pipeline(pure_dataset)
X = preproc_dataset[:train_modified.shape[0]].copy()
y = target_col
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y,
                                                  test_size=0.2,
                                                  random_state=42)


In [249]:
%%time
model = RandomForestRegressor(random_state=42)
values = [200,220,250,270,290]
selected_features_v3 = {}
for value in values:
    rfe = RFE(estimator=model, n_features_to_select=value)
    rfe.fit(X_train, y_train)
    selected_features_v3[value] = X_train.columns[rfe.support_]

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\sklearn\feature_selection\_rfe.py:300: UserWarning: Found n_features_to_select=290 > n_features=286. There will be no feature selection and all features will be kept.
  warnings.warn(


CPU times: total: 9min 37s
Wall time: 9min 47s


In [250]:
path = r"C:\Users\Adam\Desktop\main\programming\machine learning\house prices regression\data\preprocessed\data3.pkl"
with open("data_feature_selection3.pkl", "wb") as path:
    pickle.dump(selected_features_v3, path)

In [251]:
dnn_rmse_feature_selection_v3 = {}
for value, features in selected_features_v3.items():
    rmse_dnn = DNN(X_train[features], y_train, X_val[features], y_val)
    dnn_rmse_feature_selection_v3[value] = rmse_dnn[-1]
    rmse_dnn

C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Adam\anaconda3\envs\housing-prices\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__i

In [252]:
dnn_rmse_feature_selection_v3

{200: 34212.2109375,
 220: 33601.3046875,
 250: 34368.71875,
 270: 33930.69921875,
 290: 34060.8828125}

In [253]:
rmse_models_feature_selection_v3 = {}
for value, features in selected_features_v3.items():
    rmse_models = evaluate_models(X_train[features], X_val[features], y_train, y_val, models)
    rmse_models_feature_selection_v3[str(value) + 'LinearRegression'] = float(rmse_models[0][2].numpy())
    rmse_models_feature_selection_v3[str(value) + 'RandomForestRegressor'] = float(rmse_models[1][2].numpy())
rmse_models_feature_selection_v3

{'200LinearRegression': 33293.765625,
 '200RandomForestRegressor': 28945.73046875,
 '220LinearRegression': 34171.02734375,
 '220RandomForestRegressor': 28325.24609375,
 '250LinearRegression': 32536.439453125,
 '250RandomForestRegressor': 28842.2578125,
 '270LinearRegression': 33492.125,
 '270RandomForestRegressor': 29221.36328125,
 '290LinearRegression': 31554.5859375,
 '290RandomForestRegressor': 29357.09765625}

In [254]:
rmse_threshold

,model,rmse
0,DNN,34363.132812
1,LinearRegression,31510.705078
2,RandomForestRegressor,28452.984375


### NOTE 
Here we can see that pure dataset works best for DNNs while modified, one with newly created features tend to be better for our ML models.                                                                                                                       

In [17]:
new_features_dataset = dataset_df.copy()

In [19]:
AgeOfHouse = dataset_df['YrSold'] - dataset_df['YearBuilt'] 
#new_features_dataset['AgeOfHouse'] = AgeOfHouse
YearsSinceRenov = dataset_df['YrSold'] - dataset_df['YearRemodAdd']
#new_features_dataset['YearsSinceRenov'] = YearsSinceRenov
TotalBathNum = dataset_df['FullBath'] + dataset_df['HalfBath'] + dataset_df['BsmtFullBath'] + dataset_df['BsmtHalfBath']
PorchSize = dataset_df['OpenPorchSF'] + dataset_df['EnclosedPorch'] + dataset_df['3SsnPorch'] + dataset_df['ScreenPorch']
#new_features_dataset['PorchSize'] = PorchSize
IsBasement = dataset_df['TotalBsmtSF'] > 0
IsFirePlace = dataset_df['Fireplaces'] > 0
IsGarage = dataset_df['GarageArea'] > 0

# add new features
new_features_dataset['AgeOfHouse'] =AgeOfHouse
new_features_dataset['YearsSinceRenov']  = YearsSinceRenov
new_features_dataset['TotalBathNum'] = TotalBathNum
new_features_dataset['PorchSize'] = PorchSize
new_features_dataset['IsBasement'] = IsBasement
new_features_dataset['IsFirePlace'] = IsFirePlace
new_features_dataset['IsGarage'] = IsGarage
new_features_dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,YrSold,SaleType,SaleCondition,AgeOfHouse,YearsSinceRenov,TotalBathNum,PorchSize,IsBasement,IsFirePlace,IsGarage
0,1,4.110874,RL,4.189655,9.042040,Pave,Reg,Lvl,AllPub,Inside,...,2008,WD,Normal,5,5,4.000000,4.127134,True,False,True
1,2,3.044522,RL,4.394449,9.169623,Pave,Reg,Lvl,AllPub,FR2,...,2007,WD,Normal,31,31,2.693147,0.000000,True,True,True
2,3,4.110874,RL,4.234107,9.328212,Pave,IR1,Lvl,AllPub,Inside,...,2008,WD,Normal,7,6,4.000000,3.761200,True,True,True
3,4,4.262680,RL,4.110874,9.164401,Pave,IR1,Lvl,AllPub,Corner,...,2006,WD,Abnorml,91,36,2.000000,9.192991,True,True,True
4,5,4.110874,RL,4.442651,9.565284,Pave,IR1,Lvl,AllPub,FR2,...,2008,WD,Normal,8,8,4.000000,4.442651,True,True,True
